In [1]:
import gurobipy as gp
from gurobipy import GRB
import scipy.io
import numpy as np
import math

In [2]:
# randomly generate the instances
patient_type = np.array([30,60,120,180,240,300,360])
# appointmentNumber = 10
# # a = np.random.choice(patient_type, size=appointmentNumber, replace=True)
# # B = 14
# a=np.array([30,60,60,60,120,60,30,60,120])

In [4]:
instances = "p1"
# p = [15,9,8,5,8,2,0]
p = [0,1,1,0,0,0,0]
a = []
for i in range(len(patient_type)):
    a.extend([patient_type[i]]*p[i])
a = np.array(a)

In [5]:
# get template of schedule
u1 = np.array([0,30,60,120,150,180,240,420])
v1 = np.array([30,60,120,150,180,240,420,600])
u2 = np.array([15,60,120,180,300,330,450,510])
v2 = np.array([45,120,180,300,330,450,510,540])
u3 = np.array([15,75,105,135,165,285,465])
v3 = np.array([45,105,135,165,285,465,585])
u4 = np.array([30,75,105,135,195])
v4 = np.array([60,105,135,195,555])
u5 = np.array([30,150,180,360])
v5 = np.array([150,180,360,600])
u6 = np.array([45,225,405])
v6 = np.array([225,405,525])
u7 = np.array([45,285,345,375,405])
v7 = np.array([285,345,375,405,525])
u8 = np.array([90,120,180,480])
v8 = np.array([120,180,480,510])
u9 = np.array([90,270,300,330])
v9 = np.array([270,300,330,570])
u10 = np.array([105,225,465,495])
v10 = np.array([225,465,495,525])
u11 = np.array([120,240,420])
v11 = np.array([240,420,600])
u12 = np.array([120,300])
v12 = np.array([300,540])
u13 = np.array([120,420])
v13 = np.array([420,540])
u14 = np.array([135,315])
v14 = np.array([315,555])

# u = [u1,u2,u3,u4,u5,u6,u7,u8,u9,u10,u11,u12,u13,u14]
# v = [v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14]
u = [u2]
v = [v2]
# B=14
B=1

n = len(a)
m = []
T = []
for i in range(len(u)):
    m.append(len(u[i]))
for i in range(len(v)):
    T.append(v[i][-1])
T = np.array(T)


In [6]:
epsilon = 0.05
M = []
for i in range(B):
    M.append(10000)
low_b = np.min(a)

In [7]:
model = gp.Model("model 2")
# model.setParam("LogFile",f"model2/log/{instances}.log")
w = model.addMVar((n,n,B),vtype = GRB.BINARY,name = "w")
z = model.addMVar((n,max(m),B),vtype = GRB.BINARY,name = "z")
y = model.addMVar((max(m),B,n),vtype = GRB.BINARY,name = "y")
x = model.addMVar((n,max(m),B),vtype = GRB.BINARY,name = "x")
g = model.addMVar((n,B),vtype = GRB.BINARY,name = "g")
h = model.addMVar((max(m),B),vtype = GRB.BINARY,name = "h")
r = model.addMVar((n,B),vtype = GRB.BINARY,name = "r")
q = model.addMVar((max(m),B,n),vtype = GRB.BINARY,name = "q")
s = model.addMVar((n,B),vtype = GRB.CONTINUOUS, name = "s")
t = model.addMVar((n,B),vtype = GRB.CONTINUOUS,name = "t")
d = model.addMVar((n,B),vtype = GRB.CONTINUOUS, name = "d")
p = model.addMVar((n,max(m),B),vtype = GRB.CONTINUOUS, name = "p")
x_o = model.addMVar((n,max(m),B),vtype = GRB.CONTINUOUS,name = "x_o")


Set parameter Username
Set parameter LicenseID to value 2598989
Academic license - for non-commercial use only - expires 2025-12-13


In [8]:
expr = gp.MLinExpr.zeros((n,max(m),B))
for i in range(n):
    for b in range(B):
        for j in range(m[b]):
#             model.addConstr(x_o[i,j,b] == p[i,j,b] / (v[b][j]-u[b][j]))
            expr[i,j,b] = p[i,j,b] / (v[b][j]-u[b][j])

In [21]:
model.addConstr(s+d <= t)
for i in range(n):
    for b in range(B):
        model.addConstr(s[i,b] >= u[b][0])
        model.addConstr(t[i,b] >= u[b][0])
        model.addConstr(t[i,b] <= T[b]+M[b]*(1-r[i,b]))
        if i >= 1:
            model.addConstr(t[i-1,b] <= s[i,b])
        model.addConstr(w[i,:,b].sum() <= 1)
        if i < n-1:
            model.addConstr(w[i,:,b].sum() >= w[i+1,:,b].sum())
        model.addConstr(w[i,:,b].sum() >= r[i,b])
        model.addConstr(d[i,b] == w[i,:,b]@a)
    model.addConstr(w[:,i,:].sum() <= 1)
for b in range(B):
    for i in range(n-1):
        model.addConstr(r[i,b] >= r[i+1,b])
for i in range(n):
    for b in range(B):
        for j in range(m[b]):
            model.addConstr(p[i,j,b] >= x[i,j,b] * low_b)
            model.addConstr(p[i,j,b] >= t[i,b] - s[i,b] + x[i,j,b]*T[b]-T[b])
            model.addConstr(p[i,j,b] <= t[i,b] - s[i,b] + x[i,j,b]*low_b-low_b)
            model.addConstr(p[i,j,b] <= x[i,j,b]*T[b])
#             model.addConstr(p[i,j,b] >= x[i,j,b] * 0)
#             model.addConstr(p[i,j,b] >= d[i,b] + x[i,j,b]*360-360)
#             model.addConstr(p[i,j,b] >= d[i,b] + x[i,j,b]*360-360)
#             model.addConstr(p[i,j,b] <= d[i,b] + x[i,j,b]*0-0)
#             model.addConstr(p[i,j,b] <= x[i,j,b]*360)

# constraints for z
for i in range(1,n):
    for b in range(B):
        for j in range(m[b]):
            model.addConstr(t[i-1,b] - epsilon >= u[b][j] - M[b]*(1-z[i,j,b]))
            model.addConstr(s[i,b] + epsilon <= v[b][j] + M[b]*(1-z[i,j,b]))
            model.addConstr(z[i,j,b] <= r[i,b])
            model.addConstr(z[i,j,b] <= r[i-1,b])
    model.addConstr(z[i,:,:].sum() <= 1)
for b in range(B):
    for j in range(m[b]):
        model.addConstr(z[0,j,b] == 0)

 

# constraints for y when y = 1
for i in range(n):
    for b in range(B):
        for j in range(1,m[b]):
            model.addConstr(s[i,b] + epsilon <= v[b][j-1] + M[b]*(1-y[j,b,i]))
            model.addConstr(t[i,b] - epsilon >= v[b][j-1] - M[b]*(1-y[j,b,i]))
            model.addConstr(y[j,b,i] <= r[i,b])
for b in range(B):
    for j in range(1,m[b]):
        model.addConstr(y[j,b,:].sum() <= 1)  
        
# constraints for y when y = 0
for i in range(n):
    for b in range(B):
        for j in range(1,m[b]):
            model.addConstr(s[i,b] + M[b]*q[j,b,i] + M[b]*y[j,b,i] >= v[b][j-1])
            model.addConstr(t[i,b] - M[b]*(1-q[j,b,i])-M[b]*y[j,b,i] <= v[b][j-1])
for i in range(n):
    for b in range(B):
        model.addConstr(y[0,b,i] == 0)
        model.addConstr(q[0,b,i] == 0)
for i in range(n):
    for b in range(B):
        for j in range(1,m[b]):
            model.addConstr(q[j,b,i] >= y[j,b,i])

# constraints for g
for i in range(1,n-1):
    for b in range(1,B):
#         temp1 = 0
#         for j in range(1,m[b]):
#             temp1 = temp1 + y[j,b,i]
        model.addConstr(g[i,b] <= z[i,:,b].sum()+z[i+1,:,b].sum()+y[1:,b,i].sum())
for b in range(B):   
#     temp2 = 0
#     for j in range(1,m[b]):
#         temp2 = temp2 + y[j,b,0]
    model.addConstr(g[0,b] <= z[1,:,b].sum() + y[1:,b,0].sum())    
for b in range(B):
#     temp3 = 0
#     for j in range(1,m[b]):
#         temp3 = temp3 + y[j,b,n-1]
    model.addConstr(g[n-1,b] <= z[n-1,:,b] + y[1:,b,n-1].sum())
for i in range (1,n):
    for b in range(B):
        model.addConstr(g[i,b] >= z[i,:,b].sum())
for i in range(n):
    for b in range(B):
        for j in range(1,m[b]):
            model.addConstr(g[i,b] >= y[j,b,i])
        model.addConstr(g[i,b] >= 0)
        model.addConstr(g[i,b] <= 1)

# constraints for h
for b in range(B):
    for j in range(1,m[b]-1):
#         temp4 = 0
#         for i in range(1,n):
#             temp4 = temp4 + z[i,j,b]
        model.addConstr(h[j,b] <= z[1:,j,b].sum() + y[j,b,:].sum() + y[j+1,b,:].sum())
        
for b in range(B):
#     temp5 = 0
#     for i in range(1,n):
#         temp5 = temp5 + z[i,0,b]
    model.addConstr(h[0,b] <= z[1:,0,b].sum() + y[1,b,:].sum())     
for b in range(B):
#     temp6 = 0
#     for i in range(1,n):
#         temp6 = temp6 + z[i,m[b]-1,b]
    model.addConstr(h[m[b]-1,b] <= z[1:,m[b]-1,b].sum() + y[m[b]-1,b,:].sum())
for b in range(B):
    for j in range(m[b]):
        for i in range(1,n):
            model.addConstr(h[j,b] >= z[i,j,b])
        model.addConstr(h[j,b] >= 0)
        model.addConstr(h[j,b] <= 1)
for b in range(B):
    for j in range(1,m[b]):
        model.addConstr(h[j,b] >= y[j,b,:].sum())


# constraints for x
for i in range(n):
    for b in range(B):
        for j in range(m[b]):
            model.addConstr(x[i,j,b] <= 1-h[j,b])
            model.addConstr(x[i,j,b] <= 1-g[i,b])
            model.addConstr(x[i,j,b] <= r[i,b])
            model.addConstr(s[i,b] >= u[b][j] - M[b]*(1-x[i,j,b]))
            model.addConstr(t[i,b] <= v[b][j] + M[b]*(1-x[i,j,b]))
        model.addConstr(x[i,:,b].sum() <= r[i,b])
        model.addConstr(x[i,:,b].sum() == r[i,b] - g[i,b])
for b in range(B):
    for j in range(m[b]):
        model.addConstr(x[:,j,b].sum() <= 1)
        
# newly added
for i in range(n):
    for b in range(B):
        for j in range(m[b]):
            model.addConstr(t[i,b] - epsilon >= u[b][j] - M[b]*(1-x[i,j,b]))
            model.addConstr(s[i,b] + epsilon <= v[b][j] + M[b]*(1-x[i,j,b]))


In [20]:
b

0

In [22]:
model.setObjective(y.sum() + z.sum()-100*r.sum()-expr.sum(),GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1507 rows, 118 columns and 3823 nonzeros
Model fingerprint: 0x34a7351c
Variable types: 38 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [8e-03, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective -202 (0.00s)
Loaded MIP start from previous solve with objective -202

Presolve removed 1439 rows and 89 columns
Presolve time: 0.01s
Presolved: 68 rows, 29 columns, 248 nonzeros
Variable types: 8 continuous, 21 integer (21 binary)

Root relaxation: cutoff, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga

In [23]:
s.X

array([[120.],
       [180.]])

In [24]:
t.X

array([[180.],
       [300.]])

In [13]:
w.X

array([[[1.],
        [0.]],

       [[0.],
        [1.]]])

In [71]:
w_arr = np.array(w.X)
z_arr = np.array(z.X)
y_arr = np.array(y.X)
x_arr = np.array(x.X)
g_arr = np.array(g.X)
h_arr = np.array(h.X)
r_arr = np.array(r.X)
q_arr = np.array(q.X)
s_arr = np.array(s.X)
t_arr = np.array(t.X)
d_arr = np.array(d.X)
p_arr = np.array(p.X)
x_o_arr = np.array(x_o.X)

In [76]:
#save result
result = {"w_arr":w_arr,"z_arr":z_arr,"y_arr":y_arr,"x_arr":x_arr,"g_arr":g_arr,
          "h_arr":h_arr,"r_arr":r_arr,"q_arr":q_arr,"s_arr":s_arr,"t_arr":t_arr,
          "d_arr":d_arr,"p_arr":p_arr,"x_o_arr":x_o_arr,"p":p}
np.save(f"model2/result/{instances}.npy",result)

In [47]:
d.X

array([[ 30.,  60.],
       [ 30.,  30.],
       [ 60.,  30.],
       [ 30.,  30.],
       [ 30.,  30.],
       [ 60.,  30.],
       [ 30.,  30.],
       [ 60.,  60.],
       [ 60.,  30.],
       [ 30.,  30.],
       [ 60., 120.],
       [ 30., 120.],
       [ 60., 240.],
       [300., 180.],
       [180., 240.],
       [240.,   0.],
       [240.,   0.],
       [300.,   0.],
       [120.,   0.],
       [240.,   0.],
       [180.,   0.],
       [240.,   0.],
       [240.,   0.],
       [180.,   0.],
       [120.,   0.],
       [120.,   0.],
       [240.,   0.],
       [180.,   0.],
       [120.,   0.],
       [120.,   0.],
       [120.,   0.],
       [ 60.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.]])

In [16]:
s_arr = np.array(s.X)
s_arr_tran = s_arr.transpose()
t_arr = np.array(t.X)
t_arr_tran = t_arr.transpose()

In [17]:
import pandas as pd

In [18]:
# generate results of time assigned
time_assign = []
for i in range(s_arr_tran.shape[0]):
    temp = {}
    temp["s"] = s_arr_tran[i]
    temp["t"] = t_arr_tran[i]
    df = pd.DataFrame(temp)

    time_assign.append(df)

In [32]:
time_assign[1]

,s,t
0,45.0,285.0
1,285.0,345.0
2,345.0,375.0
3,375.0,405.0
4,405.0,525.0
5,525.0,1515.0
6,1515.0,1523.3
7,1523.3,1523.3
8,1523.3,2235.0


In [19]:
d_arr = np.array(d.X)
d_arr_tran = d_arr.transpose()

In [20]:
d_arr_tran

array([[ 60.,  60.,  60., 120.,   0.,   0.,   0.,   0.,   0.],
       [ 60.,  60.,  30.,  30., 120.,   0.,   0.,   0.,   0.]])

In [21]:
count = {0:0,30:0,60:0,120:0,180:0,240:0,300:0,360:0}
for bed in range(B):
    for i in range(len(a)):
        count[d_arr_tran[bed,i]] += 1 

In [22]:
count

{0: 9, 30: 2, 60: 5, 120: 2, 180: 0, 240: 0, 300: 0, 360: 0}

In [23]:
# find combined timeslots for an appointment
y_arr = np.array(y.X)

y_nonzero_idx = np.nonzero(y_arr)

y_nonzero_idx = list(zip(y_nonzero_idx[0], y_nonzero_idx[1],y_nonzero_idx[2]))

y_nonzero_idx #(j,b,i)

[]

In [27]:
# # find combined timeslots for an appointment
# x_arr = np.array(x.X)

# x_nonzero_idx = np.nonzero(x_arr)

# x_nonzero_idx = list(zip(x_nonzero_idx[0], x_nonzero_idx[1],x_nonzero_idx[2]))

# len(x_nonzero_idx)#(i,j,b)

9

In [24]:
# find the number of timeslots for policy 3 which is assigned to a smaller appointment

In [28]:
p_arr = np.array(p.X)
policy3 = 0
for i in range(n):
    for b in range(B):
        for j in range(m[b]):
            if d_arr[i,b]!= 0 and x_arr[i,j,b] != 0:
                temp = d_arr[i,b] / (v[b][j]-u[b][j])
                if temp != 1:
                    print(i,j,b)
                    policy3 += 1 

0 0 1
2 6 0
3 7 0


In [29]:
policy3

3

In [ ]:
d_arr.shape

In [ ]:
p_arr[0,:,0].sum()

In [ ]:
d_arr[0,0]

In [ ]:
# find combined timeslots for an appointment
z_arr = np.array(z.X)

z_nonzero_idx = np.nonzero(z_arr)

z_nonzero_idx = list(zip(z_nonzero_idx[0], z_nonzero_idx[1],z_nonzero_idx[2]))

len(z_nonzero_idx)#(i,j,b)

In [ ]:
d_arr[0,0]

In [ ]:
p_arr[4,:,3]

In [ ]:
x_arr[0,:,0]

In [ ]:
h_arr = np.array(h.X)
h_arr[0,0]


In [ ]:
s_arr[0,0]

In [ ]:
v[0][0]

In [ ]:
x_arr[0,0,0]

In [ ]:
epsilon

In [30]:
s_arr[0,0] + epsilon <= v[0][0] + M[0]*(1-x_arr[0,0,0])

True